# Desenvolvimento da Carteira Consolidada

**Preparado por:** Vinicius Suguimoto

In [2]:
import os
import sys

module_path = os.path.abspath(os.path.join('..'))

if module_path not in sys.path:
    sys.path.append(module_path)

import aurum
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

In [3]:
# Teoricamente toda essa etapa de treinamento do modelo já estará encapsulada no
# modelo salvo na pasta models, contudo a partir desse diretório não consigo acessar
# a pasta. Treinaremos um modelo simples para consolidar os resultados

def treinar_modelo(TICKER):
    DATA = aurum.data.get_ohlcv(TICKER)
    DATA = aurum.ft.technical_indicators(DATA)

    TRAIN_SIZE = int(len(DATA) * .75)

    INDICADORES = [
        'RSI_14',
        'STOCHk_14_3_3',
        'STOCHd_14_3_3',
        'ROC_2',
        'ROC_5',
        'ROC_10',
        'SLOPE_3',
        'ATRr_5',
        'WILLR_14',
        'OBV',
        'OBV_ROC_14',
        'EMA_BUY_CROSS',
        'EMA_SELL_CROSS',
        'EMA_9_DISTANCE', 
        'EMA_21_DISTANCE',
        'BBAND_FechouFora_Lower',
        'BBAND_FechouFora_Upper'
    ]

    X_train = DATA.loc[TRAIN_SIZE:, INDICADORES]
    y_train = DATA.loc[TRAIN_SIZE:, 'Alvo']

    MODEL = aurum.model_training.create_setup(X_train, y_train)

    return MODEL

# Modelos para a Carteira

MODELO_PETR4 = treinar_modelo('PETR4.SA')
MODELO_CMIG3 = treinar_modelo('CMIG3.SA')
MODELO_ABEV3 = treinar_modelo('ABEV3.SA')

In [4]:
# Resultados consolidados

def model_results(TICKER, MODEL): 

    TEST = aurum.data.get_ohlcv(TICKER, TREINO=False)
    TEST = aurum.ft.technical_indicators(TEST)
    TEST['Predicao'] = MODEL.predict(TEST[MODEL.feature_names_in_])

    TEST['Ticker'] = TICKER
    TEST['Retorno do Modelo'] = TEST['Predicao'] * TEST['LEAK_Retorno']/5

    RETURN = TEST[['Ticker', 'Date', 'Predicao', 'Retorno do Modelo']]

    return RETURN



In [5]:
RETORNO_PETR4 = model_results('PETR4.SA', MODELO_PETR4)
RETORNO_CMIG3 = model_results('CMIG3.SA', MODELO_CMIG3)
RETORNO_ABEV3 = model_results('ABEV3.SA', MODELO_ABEV3)

In [6]:
def wallet_return(RETURN_LIST, TICKER_WEIGHT):

    assert len(RETURN_LIST) == len(TICKER_WEIGHT)
    assert sum(TICKER_WEIGHT) == 1

    
    RETORNO_PONDERADO_MODELO = np.zeros(len(RETURN_LIST[0]))

    for i, retorno in enumerate(RETURN_LIST):

        RETORNO_PONDERADO_MODELO += retorno['Retorno do Modelo'] * TICKER_WEIGHT[i]

    RETORNO_ACUMULADO_MODELO = (1 + RETORNO_PONDERADO_MODELO).cumprod()

    RESULTADOS = pd.DataFrame(RETORNO_ACUMULADO_MODELO)
    RESULTADOS['Date'] = RETURN_LIST[0]['Date']
    RESULTADOS = RESULTADOS.rename({'Retorno do Modelo': 'Retorno da Carteira'}, axis=1)

    return RESULTADOS[['Date', 'Retorno da Carteira']]

In [7]:
RETORNO_GERAL = wallet_return([RETORNO_PETR4, RETORNO_CMIG3], [.5, .5])

In [8]:
import plotly.express as px

In [9]:
px.line(RETORNO_GERAL,x='Date', y='Retorno da Carteira')

---

In [10]:
lista_retornos = [RETORNO_PETR4, RETORNO_CMIG3, RETORNO_ABEV3]
consolidado = pd.concat(lista_retornos)
consol_pivot = consolidado.pivot_table(index='Date', columns=['Ticker'], values=['Predicao', 'Retorno do Modelo']).tail(10)

In [11]:
# Retorno das posições abertas 5 dias atrás
POSICAO_EM_FECHAMENTO = consol_pivot.iloc[-6]
RESULT_ULT_POSICAO = POSICAO_EM_FECHAMENTO['Retorno do Modelo'][POSICAO_EM_FECHAMENTO.Predicao == 1].reset_index()

In [15]:
RESULT_ULT_POSICAO

,Ticker,2022-11-07 00:00:00
0,ABEV3.SA,-0.005012
1,PETR4.SA,0.004052


In [12]:
# Últimas entradas
ULTIMA_ABERTURA = consol_pivot.iloc[-1].Predicao.reset_index()
ULTIMA_ABERTURA['Sinal'] = ['Compra' if x == 1 else '-' for x in ULTIMA_ABERTURA.iloc[:, -1]]
#ULTIMA_ABERTURA.Sinal

In [14]:
ULTIMA_ABERTURA.columns[1]

Timestamp('2022-11-14 00:00:00')

In [ ]:
class AurumModel:

    def __init__(self, NOME_MODELO):
        self.NOME_MODELO = NOME_MODELO
        self.TICKER = pd.Series(NOME_MODELO).str.extract('- (.*.).sav').values[0][0]
        self.MODELO = self.load_model(self.NOME_MODELO)
        self.FEATURES_MODELO = self.MODELO.feature_names_in_

        self.TICKER_TEST_DATA = self.load_ticker_test_data(self.TICKER)
        self.MODEL_PREDICTIONS = aurum.model_training.make_predictions(self.TICKER)